<a href="https://colab.research.google.com/github/eabdelmoneim/colab-workbooks/blob/main/Hoth_Industries_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hoth Industries: Playing Offense not Defense with AI Intelligence
**Project Objective:** Transition Hoth Industries from a fragmented, "tribal knowledge" sourcing model to an integrated, data-driven intelligence ecosystem.

## The Strategy: AI Based Risk Alerts
We leverage **CADDi Drawer** as the primary anchor for geometric and similarity intelligence, linking it to Hoth's messy internal data to solve:
1. **Sourcing Bottlenecks:** Identifying high-risk for rejection suppliers.
2. **Design Rework:** Surfacing "Design-for-Manufacturability" (DfM) risks early.
3. **Price Guardrails:** Benchmarking new quotes against historical purchase prices to prevent overpaying.

# Step 1 - Data Cleansing & Normalization

In [1]:
# --- PHASE 1: HOTH INDUSTRIES DATA CLEANSING ---
# Narrative: Addressing the lack of referential integrity in legacy Hoth systems.

import pandas as pd
import numpy as np

# Load Hoth's raw internal data (Upload these files to your Colab 'Files' tab first)
try:
    orders = pd.read_csv('Copy of supplier_orders.csv')
    quality = pd.read_csv('Copy of quality_inspections.csv')
    rfq = pd.read_csv('Copy of rfq_responses.csv')
except FileNotFoundError:
    print("Please upload the Hoth CSV files to the Colab sidebar.")

# 1. Supplier Name Normalization
# Fixing inconsistencies like 'APEX MFG' vs 'Apex Manufacturing Inc'
def normalize_supplier(name):
    return str(name).upper().replace('INC', '').replace('LLC', '').replace('SYSTEMS', 'SYS').strip()

orders['supplier_key'] = orders['supplier_name'].apply(normalize_supplier)
rfq['supplier_key'] = rfq['supplier_name'].apply(normalize_supplier)

# 2. Engineering Supply Chain Latency KPIs
orders['promised_date'] = pd.to_datetime(orders['promised_date'])
orders['actual_delivery_date'] = pd.to_datetime(orders['actual_delivery_date'])
orders['days_late'] = (orders['actual_delivery_date'] - orders['promised_date']).dt.days.fillna(0)

print("✓ PHASE 1 COMPLETE: Hoth raw data standardized.")
orders[['order_id', 'supplier_name', 'supplier_key', 'days_late']].head()

✓ PHASE 1 COMPLETE: Hoth raw data standardized.


,order_id,supplier_name,supplier_key,days_late
0,PO-2021-011,Precision Thermal Co,PRECISION THERMAL CO,0.0
1,PO-2021-011,TitanForge LLC,TITANFORGE,-1.0
2,PO-2021-011,APEX MFG,APEX MFG,-1.0
3,PO-2021-011,AeroFlow Systems,AEROFLOW SYS,0.0
4,PO-2021-011,Apex Mfg,APEX MFG,-1.0


# Step 2 - The Unified Data Warehouse

In [2]:
# --- PHASE 2: HOTH INDUSTRIES UNIFIED DATA WAREHOUSE ---
# Narrative: Anchoring Hoth's execution data to CADDi Drawer's geometric intelligence.

# Ingesting Geometric Metadata extracted via CADDi Drawer API (Mocked for Demo)
# This includes Geometry Type, Complexity Score, and Materials
drawer_meta = pd.read_csv('mock_drawer_metadata.csv')

def build_hoth_warehouse(orders, quality, drawer_meta):
    # Construct the 'Silver' Layer: Master Parts Record
    # We join Hoth's order/quality history with CADDi's Drawing Metadata
    warehouse_df = pd.merge(orders, drawer_meta, on='part_number', how='left')
    warehouse_df = pd.merge(warehouse_df, quality, on='order_id', how='left')

    # Standardize quality reporting
    warehouse_df['parts_rejected'] = warehouse_df['parts_rejected'].fillna(0)
    warehouse_df['parts_inspected'] = warehouse_df['parts_inspected'].fillna(0)

    return warehouse_df

master_warehouse = build_hoth_warehouse(orders, quality, drawer_meta)
print(f"✓ PHASE 2 COMPLETE: Warehouse built with {len(master_warehouse)} linked records.")
master_warehouse[['part_number', 'geometry_type', 'complexity_score', 'parts_rejected']].head()

✓ PHASE 2 COMPLETE: Warehouse built with 2089 linked records.


,part_number,geometry_type,complexity_score,parts_rejected
0,CTRL-9998,Electronic - Assembly,7,0
1,CTRL-9998,Electronic - Assembly,7,1
2,CTRL-9998,Electronic - Assembly,7,3
3,HX-5530,Machined - Complex,7,0
4,HX-5530,Machined - Complex,7,1


# Step 3 - Hoth Industries Drawer Intelligence Apps

In [6]:
# --- PHASE 3: HOTH INDUSTRIES STRATEGIC APPLICATIONS ---
# Narrative: Functional tools that leverage the Warehouse to provide immediate ROI.

# 1. APP: Supplier Reliability Scorecard
# Ranks suppliers by rejection rates and delivery speed for specific geometry types.
scorecard = master_warehouse.groupby(['supplier_key', 'geometry_type']).agg({
    'parts_rejected': 'sum',
    'parts_inspected': 'sum',
    'days_late': 'mean'
}).reset_index()
scorecard['rejection_rate'] = (scorecard['parts_rejected'] / scorecard['parts_inspected']).fillna(0)

# 2. APP: Geometric DfM Advisor
# Uses Drawer similarity matching to warn engineers about historical quality risks.
drawer_sim = pd.read_csv('mock_drawer_similarity.csv')

def get_hoth_dfm_advice(target_part):
    # Retrieve top geometric match from CADDi Archive
    match = drawer_sim[drawer_sim['source_part_number'] == target_part].sort_values('similarity_score', ascending=False)
    if match.empty: return "New Geometry: No similar data in Hoth archive."

    similar_part = match.iloc[0]['similar_part_number']
    score = match.iloc[0]['similarity_score']

    # Pull historical rejection rate for that geometry
    hist = master_warehouse[master_warehouse['part_number'] == similar_part]
    if hist.empty: return f"Geometric match: {similar_part} ({int(score*100)}% match). No history."

    avg_rej = hist['parts_rejected'].sum() / hist['parts_inspected'].sum()
    return f"DESIGN ALERT: {int(score*100)}% match to Part {similar_part}. Historical Rejection Rate: {avg_rej:.1%}"

# 3. APP: Quote Benchmarking Tool
# Flags new quotes that deviate significantly from historical purchase prices.
def benchmark_hoth_quote(part_num):
    # Calculate historical price from the Warehouse
    history = master_warehouse[master_warehouse['part_number'] == part_num]
    if history.empty: return "New Part: No historical price benchmark available."

    avg_paid = history['unit_price'].mean()

    # Check current RFQ responses for the same part
    latest_quote = rfq[rfq['part_description'].str.contains(part_num, na=False)]['quoted_price'].mean()

    if pd.isna(latest_quote):
        return f"Benchmark: Historical Avg = ${avg_paid:.2f} | No current RFQs."

    variance = ((latest_quote - avg_paid) / avg_paid) * 100

    if variance > 10:
        return f"PRICE ALERT: Current quote is {variance:.1f}% ABOVE historical average (${avg_paid:.2f})."
    return f"Quote status: Within 10% of historical average (${avg_paid:.2f})."

print("✓ PHASE 3 COMPLETE: Hoth Industries Applications Live.")

✓ PHASE 3 COMPLETE: Hoth Industries Applications Live.


# "Hoth Industries" Demo

In [7]:
# --- HOTH INDUSTRIES LIVE DEMO ---

# SCENARIO: An engineer is reviewing a high-impact design (HX-5530)
target_item = 'HX-5530'

print("="*60)
print(f"HOTH INDUSTRIES ANALYSIS: PART {target_item}")
print("="*60)

# Insight 1: Design Feedback
print(f"[DfM Advisor]: {get_hoth_dfm_advice(target_item)}")

# Insight 2: Sourcing Benchmarking
print(f"[Quote Benchmark]: {benchmark_hoth_quote(target_item)}")

# Insight 3: Sourcing Strategy
# Highlight the QuickFab Industries bottleneck for complex parts
try:
    risk = scorecard[scorecard['rejection_rate'] > 0.05].iloc[0]
    print(f"[Sourcing Alert]: High risk detected with {risk['supplier_key']} for {risk['geometry_type']} parts.")
except:
    print("[Sourcing Alert]: No critical risks detected for this cluster.")

HOTH INDUSTRIES ANALYSIS: PART HX-5530
[DfM Advisor]: DESIGN ALERT: 95% match to Part FINS-7712. Historical Rejection Rate: 7.1%
[Quote Benchmark]: Benchmark: Historical Avg = $1817.01 | No current RFQs.
[Sourcing Alert]: High risk detected with APEX MANUFACTURING for Electronic - Assembly parts.
